# Boston-Big Data Exam
Mayur Kolki

PGA14

## Q2. Read the data from Hive table as spark dataframe 

In [1]:
# Import pySpark in anaconda
import pandas as pd
from pyspark.sql import SparkSession

# Create spark session object
spark=SparkSession.builder.appName('lin_reg').getOrCreate()

# Import Linear Regression from MLib
from pyspark.ml.regression import LinearRegression

In [2]:
# Load Dataset

df = spark.read.csv('file:///home/hduser/demo_1/boston.csv',inferSchema=True, header=True)

In [3]:
# Check the shape of dataset
# Rows              Columns
print((df.count(), len(df.columns)))

(506, 14)


## Q3. Get the correlation between dependent and independent variables

In [5]:
#import corr function/library from pyspark functions

from pyspark.sql.functions import corr

In [6]:
df.select(corr('CRIM','MV')).show() ; df.select(corr('ZN','MV')).show()

+-------------------+
|     corr(CRIM, MV)|
+-------------------+
|-0.3883046116575089|
+-------------------+

+-----------------+
|     corr(ZN, MV)|
+-----------------+
|0.360445344637529|
+-----------------+



## Q4. Build a linear regression model to predict house price

In [12]:
#import vectorassembler to create dense vectors

from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler

In [13]:
#select the columns to create input vector
df.columns

['CRIM',
 'ZN',
 'INDUS',
 'CHAS',
 'NOX',
 'RM',
 'AGE',
 'DIS',
 'RAD',
 'TAX',
 'PT',
 'B',
 'LSTAT',
 'MV']

In [14]:
#create the vector assembler
vec_assmebler=VectorAssembler(inputCols=['CRIM','ZN','INDUS','CHAS','NOX','RM','AGE','DIS','RAD','TAX','PT','B','LSTAT',],outputCol='features')

In [15]:
#transform the values
features_df=vec_assmebler.transform(df)

In [16]:
#validate the presence of dense vectors
features_df.printSchema()

root
 |-- CRIM: double (nullable = true)
 |-- ZN: double (nullable = true)
 |-- INDUS: double (nullable = true)
 |-- CHAS: integer (nullable = true)
 |-- NOX: double (nullable = true)
 |-- RM: double (nullable = true)
 |-- AGE: double (nullable = true)
 |-- DIS: double (nullable = true)
 |-- RAD: integer (nullable = true)
 |-- TAX: integer (nullable = true)
 |-- PT: double (nullable = true)
 |-- B: double (nullable = true)
 |-- LSTAT: double (nullable = true)
 |-- MV: double (nullable = true)
 |-- features: vector (nullable = true)



In [17]:
#view the details of dense vector
features_df.select('features').show(5,False)

+-------------------------------------------------------------------------------------------------------------------------------+
|features                                                                                                                       |
+-------------------------------------------------------------------------------------------------------------------------------+
|[0.00632,18.0,2.309999943,0.0,0.537999988,6.574999809,65.19999695,4.090000153,1.0,296.0,15.30000019,396.8999939,4.980000019]   |
|[0.027310001,0.0,7.070000172,0.0,0.469000012,6.421000004,78.90000153,4.967100143,2.0,242.0,17.79999924,396.8999939,9.140000343]|
|[0.02729,0.0,7.070000172,0.0,0.469000012,7.184999943,61.09999847,4.967100143,2.0,242.0,17.79999924,392.8299866,4.03000021]     |
|[0.032370001,0.0,2.180000067,0.0,0.458000004,6.998000145,45.79999924,6.062200069,3.0,222.0,18.70000076,394.6300049,2.940000057]|
|[0.069049999,0.0,2.180000067,0.0,0.458000004,7.146999836,54.20000076,6.062200069,3.0,222.

In [18]:
#create data containing input features and output column
model_df=features_df.select('features','MV')

In [19]:
model_df.show(5,False)

+-------------------------------------------------------------------------------------------------------------------------------+-----------+
|features                                                                                                                       |MV         |
+-------------------------------------------------------------------------------------------------------------------------------+-----------+
|[0.00632,18.0,2.309999943,0.0,0.537999988,6.574999809,65.19999695,4.090000153,1.0,296.0,15.30000019,396.8999939,4.980000019]   |24.0       |
|[0.027310001,0.0,7.070000172,0.0,0.469000012,6.421000004,78.90000153,4.967100143,2.0,242.0,17.79999924,396.8999939,9.140000343]|21.60000038|
|[0.02729,0.0,7.070000172,0.0,0.469000012,7.184999943,61.09999847,4.967100143,2.0,242.0,17.79999924,392.8299866,4.03000021]     |34.70000076|
|[0.032370001,0.0,2.180000067,0.0,0.458000004,6.998000145,45.79999924,6.062200069,3.0,222.0,18.70000076,394.6300049,2.940000057]|33.40000153|
|[0.06

In [20]:
# shape of model df
print((model_df.count(), len(model_df.columns)))

(506, 2)


In [21]:
## Split Data - Train & Test sets

#split the data into 70/30 ratio for train test purpose
train_df,test_df=model_df.randomSplit([0.7,0.3])

In [22]:
# Dimension of Train data

print((train_df.count(), len(train_df.columns)))

(357, 2)


In [23]:
# Dimension of Test data

print((test_df.count(), len(test_df.columns)))

(149, 2)


In [25]:
#Build Linear Regression model

lin_Reg=LinearRegression(labelCol='MV')

In [26]:
#fit the linear regression model on training data set

lr_model=lin_Reg.fit(train_df)

In [ ]:
# Prediction 

training_predictions=lr_model.evaluate(train_df)

In [85]:
# Training MSE

training_predictions.meanSquaredError

21.777646447204766

In [86]:
# R Square

training_predictions.r2

0.727370137782524

## Q5. Evaluate the Linear Regression model by getting the RMSE and R-squared values

In [87]:
#make predictions on test data
test_results=lr_model.evaluate(test_df)

In [90]:
#coefficient of determination value for model

test_results.r2

0.7444034376599911

In [91]:
# RMSE - Root Mean Square Error

test_results.rootMeanSquaredError

4.890333189486392

In [92]:
test_results.meanSquaredError

23.915358704192148